# Basic Classification Systems
Por Enrique Juliá Arévalo, Sara Verde Camacho y Leo Pérez Peña

El objetivo de esta práctica es comparar el error de predicción de los siguientes clasificadores:
- Naive Bayes Classifier
- Linear Discriminant Analysis (LDA)
- Quadratic Discriminant Analysis (QDA)
- Nearest Shrunken Centroids Classifier

Utilizando el conjunto de datos de cáncer de mama wdbc.csv y el conjunto de datos de cáncer de próstata prostate.csv.

In [32]:
# Cargamos las librerías necesarias
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import NearestCentroid
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, make_scorer
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

En primer lugar, cargamos los datos y extraemos las dimensiones de las tablas para hacernos una idea de cuántos datos tenemos en cada caso.

In [2]:
breast_data = pd.read_csv('wdbc.csv', header=None)
breast_data.shape

(569, 32)

In [3]:
prostate_data = pd. read_csv('prostate.csv')
prostate_data.shape

(102, 12626)

En el primer caso, el número de datos es mucho mayor que el número de dimensiones. Sin embargo, en el segundo caso, el número de dimensiones es mucho mayor que el número de datos. Esto probablemente afectará al rendimiento de los distintos clasificadores, ya que unos clasificadores son más adecuados en un caso y otros en otro.

Una vez cargados los datos, extraemos las etiquetas.

En el primer conjunto de datos, las etiquetas están en la segunda columna:

In [4]:
breast_data

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [6]:
X_breast = breast_data.values[:, 2:].astype(float)
y_breast = (breast_data.values[:, 1] == 'M').astype(int) # 1 para maligno, 0 para benigno
print(y_breast)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1
 0 1 0 1 1 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 1 0 1 1 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1
 0 1 0 0 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0
 0 1 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0
 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0
 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 1 0 1 0 0 1 

En el segundo conjunto de datos, las estiquetas están en la última columna:

In [7]:
prostate_data

,100_g_at,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,...,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,Y
0,6.927460,7.391657,3.812922,3.453385,6.070151,5.527153,5.812353,3.167275,7.354981,9.419909,...,3.770583,2.884436,2.730025,3.126168,2.870161,3.082210,2.747289,3.226588,3.480196,0
1,7.222432,7.329050,3.958028,3.407226,5.921265,5.376464,7.303408,3.108708,7.391872,10.539579,...,3.190759,2.460119,2.696578,2.675271,2.940032,3.126269,3.013745,3.517859,3.428752,1
2,6.776402,7.664007,3.783702,3.152019,5.452293,5.111794,7.207638,3.077360,7.488371,6.833428,...,3.325183,2.603014,2.469759,2.615746,2.510172,2.730814,2.613696,2.823436,3.049716,0
3,6.919134,7.469634,4.004581,3.341170,6.070925,5.296108,8.744059,3.117104,7.203028,10.400557,...,3.625057,2.765521,2.681757,3.310741,3.197177,3.414182,3.193867,3.353537,3.567482,0
4,7.113561,7.322408,4.242724,3.489324,6.141657,5.628390,6.825370,3.794904,7.403024,10.240322,...,3.698067,3.026876,2.691670,3.236030,3.003906,3.081497,2.963307,3.472050,3.598103,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,6.819913,7.394450,3.617486,3.032897,5.414759,5.254673,8.785910,3.671019,7.670415,9.357480,...,3.922944,2.782813,2.520608,2.885149,2.783539,2.554428,2.733469,3.068130,2.953872,1
98,7.106082,7.465860,4.091138,3.402607,5.894284,5.206407,8.796626,3.154433,7.724618,7.752833,...,3.491258,2.830696,2.688828,3.054819,2.859602,2.883547,2.651160,3.306874,3.234820,0
99,6.907847,7.076667,3.998270,3.204424,5.912380,5.486270,6.133782,3.874307,7.153055,10.432679,...,3.512675,2.590794,2.384583,2.890110,2.785744,2.755482,2.731221,3.617421,3.255625,1
100,6.767213,7.170628,3.520863,3.413946,6.294376,5.528486,5.160356,3.367304,7.022459,11.443175,...,3.654453,3.125644,2.593132,3.655418,2.668599,2.901230,2.651258,3.174388,3.263090,1


In [10]:
X_prostate = prostate_data.values[:,:-1].astype(float)
y_prostate = prostate_data.values[:, -1].astype(int)
print(y_prostate)

[0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 1 0 0 0 0 0 1 1 0 0 1 1 0
 1 0 1 0 0 0 1 1 1 0 1 1 0 1 1 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 0 1 0 1 1
 1 1 0 0 1 1 0 0 1 1 0 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 1 1]


## División de los datos en conjuntos de entrenamiento y prueba

De cada conjunto de datos, dos tercios se emplearán para entrenar los modelos y el tercio restante se utilizará para testarlos. Además, realizaremos 20 particiones diferentes, de forma que el error de predicción será la media de los errores obtenidos con cada una de ellas.

In [11]:
breast_complete = []
for i in range(20):
    bre_X_train, bre_X_test, bre_y_train, bre_y_test = train_test_split( \
        X_breast, y_breast, test_size= 1/3, random_state=i)
    breast_complete.append([bre_X_train, bre_X_test, bre_y_train, bre_y_test])
    print(f'Partition {i+1} done')

Partition 1 done
Partition 2 done
Partition 3 done
Partition 4 done
Partition 5 done
Partition 6 done
Partition 7 done
Partition 8 done
Partition 9 done
Partition 10 done
Partition 11 done
Partition 12 done
Partition 13 done
Partition 14 done
Partition 15 done
Partition 16 done
Partition 17 done
Partition 18 done
Partition 19 done
Partition 20 done


In [12]:
prostate_complete = []
for i in range(20):
    pro_X_train, pro_X_test, pro_y_train, pro_y_test = train_test_split(\
        X_prostate, y_prostate, test_size= 1/3, random_state=i)
    prostate_complete.append([pro_X_train, pro_X_test, pro_y_train, pro_y_test])
    print(f'Partition {i+1} done')

Partition 1 done
Partition 2 done
Partition 3 done
Partition 4 done
Partition 5 done
Partition 6 done
Partition 7 done
Partition 8 done
Partition 9 done
Partition 10 done
Partition 11 done
Partition 12 done
Partition 13 done
Partition 14 done
Partition 15 done
Partition 16 done
Partition 17 done
Partition 18 done
Partition 19 done
Partition 20 done


## Normalización de los datos

A continuación, transformamos los datos de manera que estos tengan una media de 0 y una desviación estándar de 1. Para ello, calculamos la media y la desviación estándar del conjunto de datos original utilizando solo los datos de entrenamiento, ya que los datos de prueba no deben ser utilizados en ninguna fase del entrenamiento del modelo.

In [13]:
# breast dataset
for i in breast_complete:
    bre_X_train, bre_X_test, bre_y_train, bre_y_test=i
    scaler = preprocessing.StandardScaler().fit(bre_X_train)
    bre_X_train_scaled = scaler.transform(bre_X_train)
    bre_X_test_scaled = scaler.transform(bre_X_test)
    i.append(bre_X_train_scaled)
    i.append(bre_X_test_scaled)

In [14]:
# prostate dataset
for i in prostate_complete:
    pro_X_train, pro_X_test, pro_y_train, pro_y_test=i
    scaler = preprocessing.StandardScaler().fit(pro_X_train)
    pro_X_train_scaled = scaler.transform(pro_X_train)
    pro_X_test_scaled = scaler.transform(pro_X_test)
    i.append(pro_X_train_scaled)
    i.append(pro_X_test_scaled)

## Naive Bayes Classifier

Una vez hemos normalizado nuestros datos, procedemos a evaluar el primer clasificador: el clasificador de Naive Bayes.

Este clasificador se basa en el teorema de Bayes, suponiendo que las características de los datos son independientes entre sí. Para cada instancia, el clasificador Naive Bayes calcula la probabilidad posterior para cada clase y asigna la clase con la probabilidad más alta.

Las principales ventajas de este clasificador son su simplicidad y robustez frente al sobreaprendizaje, sin embargo, puede cometer errores cuando sí existe una relación de dependencia entre las características de los datos. 

In [15]:
# Creamos el clasificador, suponiendo que las características siguen una distribución normal
nb = GaussianNB()

Para el conjunto de datos de cáncer de mama:

In [16]:
pred_accuracy=[]
for i in breast_complete: # Para cada partición
    bre_X_train, bre_X_test, bre_y_train, bre_y_test, bre_X_train_scaled, bre_X_test_scaled=i
    # Entrenamos el clasificador con los datos de entrenamiento normalizados
    nb.fit(bre_X_train_scaled, bre_y_train)
    # Predecimos la clase de los datos de test
    bre_y_pred = nb.predict(bre_X_test_scaled)
    # Calculamos la matriz de confusión
    conf = confusion_matrix(bre_y_test, bre_y_pred)
    TN = conf[0][0]
    TP = conf[1][1]
    FP = conf[0][1]
    FN = conf[1][0]
    # Calculamos la precisión de la predicción y la añadimos a la lista pred_accuracy
    pred_accuracy.append(((TP + TN) / (TN + TP + FP + FN)))
    print(f'Partition done. Prediction accuracy: {((TP + TN) / (TN + TP + FP + FN))}')

pred_accuracy = np.array(pred_accuracy)
print(F"The average prediction accuracy is {pred_accuracy.mean()} (SD = {pred_accuracy.std()})")

Partition done. Prediction accuracy: 0.9
Partition done. Prediction accuracy: 0.9368421052631579
Partition done. Prediction accuracy: 0.9315789473684211
Partition done. Prediction accuracy: 0.9526315789473684
Partition done. Prediction accuracy: 0.9368421052631579
Partition done. Prediction accuracy: 0.9315789473684211
Partition done. Prediction accuracy: 0.9526315789473684
Partition done. Prediction accuracy: 0.9526315789473684
Partition done. Prediction accuracy: 0.9315789473684211
Partition done. Prediction accuracy: 0.9315789473684211
Partition done. Prediction accuracy: 0.9421052631578948
Partition done. Prediction accuracy: 0.9421052631578948
Partition done. Prediction accuracy: 0.9210526315789473
Partition done. Prediction accuracy: 0.9157894736842105
Partition done. Prediction accuracy: 0.9526315789473684
Partition done. Prediction accuracy: 0.9368421052631579
Partition done. Prediction accuracy: 0.9421052631578948
Partition done. Prediction accuracy: 0.9473684210526315
Partiti

Para el conjunto de datos de cáncer de próstata:

In [17]:
pred_accuracy=[]
for i in prostate_complete: # Para cada partición
    pro_X_train, pro_X_test, pro_y_train, pro_y_test, pro_X_train_scaled, pro_X_test_scaled=i
    # Entrenamos el clasificador con los datos de entrenamiento normalizados
    nb.fit(pro_X_train_scaled, pro_y_train)
    # Predecimos la clase de los datos de test
    pro_y_pred = nb.predict(pro_X_test_scaled)
    # Calculamos la matriz de confusión
    conf = confusion_matrix(pro_y_test, pro_y_pred)
    TN = conf[0][0]
    TP = conf[1][1]
    FP = conf[0][1]
    FN = conf[1][0]
    # Calculamos la precisión de la predicción y la añadimos a la lista pred_accuracy
    pred_accuracy.append(((TP + TN) / (TN + TP + FP + FN)))
    print(f'Partition done. Prediction accuracy: {((TP + TN) / (TN + TP + FP + FN))}')

pred_accuracy = np.array(pred_accuracy)
print(F"The average prediction accuracy is {pred_accuracy.mean()} (SD = {pred_accuracy.std()})")

Partition done. Prediction accuracy: 0.6764705882352942
Partition done. Prediction accuracy: 0.8235294117647058
Partition done. Prediction accuracy: 0.4411764705882353
Partition done. Prediction accuracy: 0.7352941176470589
Partition done. Prediction accuracy: 0.7058823529411765
Partition done. Prediction accuracy: 0.7647058823529411
Partition done. Prediction accuracy: 0.5882352941176471
Partition done. Prediction accuracy: 0.7058823529411765
Partition done. Prediction accuracy: 0.7647058823529411
Partition done. Prediction accuracy: 0.5
Partition done. Prediction accuracy: 0.5588235294117647
Partition done. Prediction accuracy: 0.4411764705882353
Partition done. Prediction accuracy: 0.6470588235294118
Partition done. Prediction accuracy: 0.7352941176470589
Partition done. Prediction accuracy: 0.5588235294117647
Partition done. Prediction accuracy: 0.5
Partition done. Prediction accuracy: 0.5294117647058824
Partition done. Prediction accuracy: 0.8529411764705882
Partition done. Predic

Como podemos observar, el clasificador Naive Bayes funciona mejor con el conjunto de datos de cáncer de mama que con el de cáncer de próstata, probablemente debido a lo que se conoce como maldición de la dimensionalidad.

Pasamos ahora a evaluar el análisis discriminante:

## Discriminant Analysis

Existen principalmente dos tipos de análisis discriminante: LDA (Linear Discriminant Analysis) y QDA (Quadratic Discriminant Analysis). 

LDA intenta encontrar una frontera lineal entre las diferentes clases basándose en la suposición de que todas las clases tienen la misma varianza-covarianza. QDA, sin embargo, deja que cada clase tenga su propia matriz de varianza-covarianza, lo que le permite modelar fronteras no lineales entre las clases.

LDA es un clasificador más sencillo, menos flexible pero más robusto frente al sobreaprendizaje.

QDA es un clasificador más complejo, más flexible pero también más propenso al sobreaprendizaje, por lo que puede no ser adecuado cuando el conjunto de datos es pequeño.

### Linear Discriminant Analysis (LDA)

In [20]:
# Creamos el clasificador
lda = LinearDiscriminantAnalysis()

Para el conjunto de datos de cáncer de mama:

In [21]:
pred_accuracy=[]
for i in breast_complete: # Para cada partición
    bre_X_train, bre_X_test, bre_y_train, bre_y_test, bre_X_train_scaled, bre_X_test_scaled=i
    # Entrenamos el clasificador con los datos de entrenamiento normalizados
    lda.fit(bre_X_train_scaled, bre_y_train)
    # Predecimos la clase de los datos de test
    bre_y_pred = lda.predict(bre_X_test_scaled)
    # Calculamos la matriz de confusión
    conf = confusion_matrix(bre_y_test, bre_y_pred)
    TN = conf[0][0]
    TP = conf[1][1]
    FP = conf[0][1]
    FN = conf[1][0]
    # Calculamos la precisión de la predicción y la añadimos a la lista pred_accuracy
    pred_accuracy.append(((TP + TN) / (TN + TP + FP + FN)))
    print(f'Partition done. Prediction accuracy: {((TP + TN) / (TN + TP + FP + FN))}')

pred_accuracy = np.array(pred_accuracy)
print(F"The average prediction accuracy is {pred_accuracy.mean()} (SD = {pred_accuracy.std()})")

Partition done. Prediction accuracy: 0.9736842105263158
Partition done. Prediction accuracy: 0.9631578947368421
Partition done. Prediction accuracy: 0.9526315789473684
Partition done. Prediction accuracy: 0.9631578947368421
Partition done. Prediction accuracy: 0.9789473684210527
Partition done. Prediction accuracy: 0.9631578947368421
Partition done. Prediction accuracy: 0.9473684210526315
Partition done. Prediction accuracy: 0.9526315789473684
Partition done. Prediction accuracy: 0.9631578947368421
Partition done. Prediction accuracy: 0.9631578947368421
Partition done. Prediction accuracy: 0.9631578947368421
Partition done. Prediction accuracy: 0.9526315789473684
Partition done. Prediction accuracy: 0.9526315789473684
Partition done. Prediction accuracy: 0.9578947368421052
Partition done. Prediction accuracy: 0.9526315789473684
Partition done. Prediction accuracy: 0.9526315789473684
Partition done. Prediction accuracy: 0.9578947368421052
Partition done. Prediction accuracy: 0.957894736

Para el conjunto de datos de cáncer de próstata:

In [22]:
pred_accuracy=[]
for i in prostate_complete: # Para cada partición
    pro_X_train, pro_X_test, pro_y_train, pro_y_test, pro_X_train_scaled, pro_X_test_scaled=i
    # Entrenamos el clasificador con los datos de entrenamiento normalizados
    lda.fit(pro_X_train_scaled, pro_y_train)
    # Predecimos la clase de los datos de test
    pro_y_pred = lda.predict(pro_X_test_scaled)
    # Calculamos la matriz de confusión
    conf = confusion_matrix(pro_y_test, pro_y_pred)
    TN = conf[0][0]
    TP = conf[1][1]
    FP = conf[0][1]
    FN = conf[1][0]
    # Calculamos la precisión de la predicción y la añadimos a la lista pred_accuracy
    pred_accuracy.append(((TP + TN) / (TN + TP + FP + FN)))
    print(f'Partition done. Prediction accuracy: {((TP + TN) / (TN + TP + FP + FN))}')

pred_accuracy = np.array(pred_accuracy)
print(F"The average prediction accuracy is {pred_accuracy.mean()} (SD = {pred_accuracy.std()})")

Partition done. Prediction accuracy: 0.9117647058823529
Partition done. Prediction accuracy: 0.8529411764705882
Partition done. Prediction accuracy: 0.7941176470588235
Partition done. Prediction accuracy: 0.8823529411764706
Partition done. Prediction accuracy: 0.8529411764705882
Partition done. Prediction accuracy: 0.8529411764705882
Partition done. Prediction accuracy: 0.8529411764705882
Partition done. Prediction accuracy: 0.8235294117647058
Partition done. Prediction accuracy: 0.8235294117647058
Partition done. Prediction accuracy: 0.9117647058823529
Partition done. Prediction accuracy: 0.7941176470588235
Partition done. Prediction accuracy: 0.8529411764705882
Partition done. Prediction accuracy: 0.8823529411764706
Partition done. Prediction accuracy: 0.8529411764705882
Partition done. Prediction accuracy: 0.9117647058823529
Partition done. Prediction accuracy: 0.7647058823529411
Partition done. Prediction accuracy: 0.7941176470588235
Partition done. Prediction accuracy: 0.823529411

Al igual que el clasificador de Naive Bayes, este clasificador funciona mejor con el conjunto de datos de cáncer de mama que con el de cáncer de próstata, también por la maldición de la dimensionalidad.

### Quadratic Discriminant Analysis (QDA)

Antes de entrenar el clasificador, tenemos que elegir un valor adecuado para el parámetro de regularización. Para ello, empleamos la técnica de grid-search, guiada por validación cruzada.

In [28]:
pipeline = Pipeline([ ('qda', QuadraticDiscriminantAnalysis()) ])

In [29]:
reg_param_values = np.linspace(0, 1, 10).tolist()
param_grid = { 'qda__reg_param': reg_param_values }

Para el conjunto de datos de cáncer de mama:

In [30]:
pred_accuracy = []

for i in breast_complete: # Para cada partición
    bre_X_train, bre_X_test, bre_y_train, bre_y_test, bre_X_train_scaled, bre_X_test_scaled = i
    
    # Buscamos el mejor valor del parámetro de regularización utilizando solo los datos de entrenamiento
    skfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=0)
    gridcv = GridSearchCV(pipeline, cv=skfold, n_jobs=1, param_grid=param_grid, \
        scoring = make_scorer(accuracy_score))
    result = gridcv.fit(bre_X_train_scaled, bre_y_train)
    accuracies = gridcv.cv_results_['mean_test_score']
    best_reg_param_value = reg_param_values[max(enumerate(accuracies), key=lambda x: x[1])[0]]
    
    # Creamos el clasificador con el mejor valor del parámetro de regularización
    qda = QuadraticDiscriminantAnalysis(reg_param = best_reg_param_value)
    # Entrenamos el clasificador con los datos de entrenamiento normalizados
    qda.fit(bre_X_train_scaled, bre_y_train)  
    # Predecimos la clase de los datos de test
    bre_y_pred = qda.predict(bre_X_test_scaled)
    # Calculamos la matriz de confusión
    conf = confusion_matrix(bre_y_test, bre_y_pred)
    TN = conf[0][0]
    TP = conf[1][1]
    FP = conf[0][1]
    FN = conf[1][0]
    # Calculamos la precisión de la predicción y la añadimos a la lista pred_accuracy
    pred_accuracy.append(((TP + TN) / (TN + TP + FP + FN)))
    print(f'Partition done. Prediction accuracy: {((TP + TN) / (TN + TP + FP + FN))}')

pred_accuracy = np.array(pred_accuracy)
print(F"The average prediction accuracy is {pred_accuracy.mean()} (SD = {pred_accuracy.std()})")

Partition done. Prediction accuracy: 0.9578947368421052
Partition done. Prediction accuracy: 0.9789473684210527
Partition done. Prediction accuracy: 0.968421052631579
Partition done. Prediction accuracy: 0.9789473684210527
Partition done. Prediction accuracy: 0.968421052631579
Partition done. Prediction accuracy: 0.968421052631579
Partition done. Prediction accuracy: 0.968421052631579
Partition done. Prediction accuracy: 0.9736842105263158
Partition done. Prediction accuracy: 0.9631578947368421
Partition done. Prediction accuracy: 0.968421052631579
Partition done. Prediction accuracy: 0.9842105263157894
Partition done. Prediction accuracy: 0.9526315789473684
Partition done. Prediction accuracy: 0.9473684210526315
Partition done. Prediction accuracy: 0.9526315789473684
Partition done. Prediction accuracy: 0.9736842105263158
Partition done. Prediction accuracy: 0.9736842105263158
Partition done. Prediction accuracy: 0.9736842105263158
Partition done. Prediction accuracy: 0.96315789473684

Para el conjunto de datos de cáncer de próstata:

In [31]:
pred_accuracy = []

for i in prostate_complete: # Para cada partición
    pro_X_train, pro_X_test, pro_y_train, pro_y_test, pro_X_train_scaled, pro_X_test_scaled = i
    
    # Buscamos el mejor valor del parámetro de regularización utilizando solo los datos de entrenamiento
    skfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=0)
    gridcv = GridSearchCV(pipeline, cv=skfold, n_jobs=1, param_grid=param_grid, \
        scoring = make_scorer(accuracy_score))
    result = gridcv.fit(pro_X_train_scaled, pro_y_train)
    accuracies = gridcv.cv_results_['mean_test_score']
    best_reg_param_value = reg_param_values[max(enumerate(accuracies), key=lambda x: x[1])[0]]

    # Creamos el clasificador con el mejor valor del parámetro de regularización
    qda = QuadraticDiscriminantAnalysis(reg_param = best_reg_param_value)
    # Entrenamos el clasificador con los datos de entrenamiento normalizados
    qda.fit(pro_X_train_scaled, pro_y_train)
    # Predecimos la clase de los datos de test
    pro_y_pred = qda.predict(pro_X_test_scaled)
    # Calculamos la matriz de confusión
    conf = confusion_matrix(pro_y_test, pro_y_pred)
    TN = conf[0][0]
    TP = conf[1][1]
    FP = conf[0][1]
    FN = conf[1][0]
    # Calculamos la precisión de la predicción y la añadimos a la lista pred_accuracy
    pred_accuracy.append(((TP + TN) / (TN + TP + FP + FN)))
    print(f'Partition done. Prediction accuracy: {((TP + TN) / (TN + TP + FP + FN))}')

pred_accuracy = np.array(pred_accuracy)
print(F"The average prediction accuracy is {pred_accuracy.mean()} (SD = {pred_accuracy.std()})")

Partition done. Prediction accuracy: 0.5588235294117647
Partition done. Prediction accuracy: 0.6764705882352942
Partition done. Prediction accuracy: 0.4411764705882353
Partition done. Prediction accuracy: 0.6764705882352942
Partition done. Prediction accuracy: 0.6470588235294118
Partition done. Prediction accuracy: 0.7058823529411765
Partition done. Prediction accuracy: 0.7647058823529411
Partition done. Prediction accuracy: 0.7941176470588235
Partition done. Prediction accuracy: 0.7941176470588235
Partition done. Prediction accuracy: 0.5
Partition done. Prediction accuracy: 0.5882352941176471
Partition done. Prediction accuracy: 0.7058823529411765
Partition done. Prediction accuracy: 0.7352941176470589
Partition done. Prediction accuracy: 0.7058823529411765
Partition done. Prediction accuracy: 0.5
Partition done. Prediction accuracy: 0.5588235294117647
Partition done. Prediction accuracy: 0.5
Partition done. Prediction accuracy: 0.47058823529411764
Partition done. Prediction accuracy:

Como suponíamos, QDA funciona bastante peor con el conjunto de datos de cáncer de próstata, ya que en este conjunto el número de datos es mucho menor que el número de dimensiones y QDA es un clasificador propenso al sobreaprendizaje. 

Sin embargo, para el conjunto de datos de cáncer de mama, QDA funciona algo mejor que LDA, ya que es un clasificador más flexible.

## Nearest Shrunken Centroids Classifier

Al igual que para QDA, antes de entrenar el clasificador tenemos que elegir valores adecuados para los hiperparámetros. Para ello, de nuevo empleamos la técnica de grid-search, guiada por validación cruzada.

In [33]:
pipeline = Pipeline([ ('nsc', NearestCentroid()) ])

In [35]:
shrinkage_param_values = np.linspace(1e-6, 8, 20).tolist()
param_grid_nsc = {'nsc__shrink_threshold': shrinkage_param_values}

Para el conjunto de datos de cáncer de mama:

In [36]:
pred_accuracy = []

for i in breast_complete: # Para cada partición
    bre_X_train, bre_X_test, bre_y_train, bre_y_test, bre_X_train_scaled, bre_X_test_scaled = i

    # Buscamos el mejor valor del hiperparámetro utilizando solo los datos de entrenamiento y creamos el clasificador
    gridcv_nsc = GridSearchCV(pipeline, cv=skfold, n_jobs=1, param_grid=param_grid_nsc, \
        scoring=make_scorer(accuracy_score))
    result_nsc = gridcv_nsc.fit(bre_X_train_scaled, bre_y_train)
    accuracies = gridcv_nsc.cv_results_['mean_test_score']
    nsc = NearestCentroid(shrink_threshold = shrinkage_param_values[np.argmax(accuracies)])

    # Entrenamos el clasificador con los datos de entrenamiento normalizados
    nsc.fit(bre_X_train_scaled, bre_y_train)
    # Predecimos la clase de los datos de test
    bre_y_pred = nsc.predict(bre_X_test_scaled)
    # Calculamos la matriz de confusión
    conf = confusion_matrix(bre_y_test, bre_y_pred)
    TN = conf[0][0]
    TP = conf[1][1]
    FP = conf[0][1]
    FN = conf[1][0]
    # Calculamos la precisión de la predicción y la añadimos a la lista pred_accuracy
    pred_accuracy.append(((TP + TN) / (TN + TP + FP + FN)))
    print(f'Partition done. Prediction accuracy: {((TP + TN) / (TN + TP + FP + FN))}')

pred_accuracy = np.array(pred_accuracy)
print(F"The average prediction accuracy is {pred_accuracy.mean()} (SD = {pred_accuracy.std()})")   

Partition done. Prediction accuracy: 0.9157894736842105
Partition done. Prediction accuracy: 0.9263157894736842
Partition done. Prediction accuracy: 0.9315789473684211
Partition done. Prediction accuracy: 0.9421052631578948
Partition done. Prediction accuracy: 0.9421052631578948
Partition done. Prediction accuracy: 0.9578947368421052
Partition done. Prediction accuracy: 0.9421052631578948
Partition done. Prediction accuracy: 0.9421052631578948
Partition done. Prediction accuracy: 0.9263157894736842
Partition done. Prediction accuracy: 0.9473684210526315
Partition done. Prediction accuracy: 0.9578947368421052
Partition done. Prediction accuracy: 0.9421052631578948
Partition done. Prediction accuracy: 0.9315789473684211
Partition done. Prediction accuracy: 0.9421052631578948
Partition done. Prediction accuracy: 0.9368421052631579
Partition done. Prediction accuracy: 0.9315789473684211
Partition done. Prediction accuracy: 0.9421052631578948
Partition done. Prediction accuracy: 0.931578947

Para el conjunto de datos de cáncer de próstata:

In [37]:
pred_accuracy = []

for i in prostate_complete: # Para cada partición
    pro_X_train, pro_X_test, pro_y_train, pro_y_test, pro_X_train_scaled, pro_X_test_scaled = i

    # Buscamos el mejor valor del hiperparámetro utilizando solo los datos de entrenamiento y creamos el clasificador
    gridcv_nsc = GridSearchCV(pipeline, cv=skfold, n_jobs=1, param_grid=param_grid_nsc, \
        scoring=make_scorer(accuracy_score))
    result_nsc = gridcv_nsc.fit(pro_X_train_scaled, pro_y_train)
    accuracies = gridcv_nsc.cv_results_['mean_test_score']
    nsc = NearestCentroid(shrink_threshold = shrinkage_param_values[ np.argmax(accuracies)])

    # Entrenamos el clasificador con los datos de entrenamiento normalizados
    nsc.fit(pro_X_train_scaled, pro_y_train)
    # Predecimos la clase de los datos de test
    pro_y_pred = nsc.predict(pro_X_test_scaled)
    # Calculamos la matriz de confusión
    conf = confusion_matrix(pro_y_test, pro_y_pred)
    TN = conf[0][0]
    TP = conf[1][1]
    FP = conf[0][1]
    FN = conf[1][0]
    # Calculamos la precisión de la predicción y la añadimos a la lista pred_accuracy
    pred_accuracy.append(((TP + TN) / (TN + TP + FP + FN)))
    print(f'Partition done. Prediction accuracy: {((TP + TN) / (TN + TP + FP + FN))}')

pred_accuracy = np.array(pred_accuracy)
print(F"The average prediction accuracy is {pred_accuracy.mean()} (SD = {pred_accuracy.std()})")

Partition done. Prediction accuracy: 0.9411764705882353
Partition done. Prediction accuracy: 0.9117647058823529
Partition done. Prediction accuracy: 0.9117647058823529
Partition done. Prediction accuracy: 0.9117647058823529
Partition done. Prediction accuracy: 0.9411764705882353
Partition done. Prediction accuracy: 0.9411764705882353
Partition done. Prediction accuracy: 0.9117647058823529
Partition done. Prediction accuracy: 0.8529411764705882
Partition done. Prediction accuracy: 0.8529411764705882
Partition done. Prediction accuracy: 0.8823529411764706
Partition done. Prediction accuracy: 0.8235294117647058
Partition done. Prediction accuracy: 0.7941176470588235
Partition done. Prediction accuracy: 0.8529411764705882
Partition done. Prediction accuracy: 0.8823529411764706
Partition done. Prediction accuracy: 0.9117647058823529
Partition done. Prediction accuracy: 0.8235294117647058
Partition done. Prediction accuracy: 0.8529411764705882
Partition done. Prediction accuracy: 0.911764705

Nuevamente, el clasificador funciona mejor con el conjunto de datos de cáncer de mama.

## PREGUNTAS
- What method performs best on each dataset in terms of prediction error?

El error de predicción puede calcularse como 1 - prediction accuracy, por lo que el clasificador que funciona mejor es aquel que tiene mayor prediction accuracy (y por tanto menor error de predicción). Teniendo esto en cuenta, el clasificador que funciona mejor con el conjunto de datos de cáncer de mama es QDA y el que funciona mejor con el conjunto de datos de cáncer de próstata es NSC.

- What method is more flexible according to what we have seen in the lectures? Is this compatible with the obtained results?

El clasificador más flexible (y que por tanto debería ajustarse mejor a los datos) es NSC que, como hemos visto, funciona bien tanto con el conjunto de datos de cáncer de mama como con el conjunto de datos de cáncer de próstata.

QDA también es flexible, aunque no funciona también con el conjunto de datos de cáncer de próstata debido al sobreaprendizaje.

- What method is more robust to over-fitting according to what we have seen in the lectures? Is this compatible with the obtained results?

Los clasificadores más robustos frente al sobreaprendizaje son los más sencillos, como Naive Bayes o LDA. Después de NSC, LDA es el clasificador que mejor funciona con el conjunto de datos de cáncer de próstata, lo que concuerda con lo que cabría esperar.

- Discuss and compare, regarding the bias and the variance, the procedure used to estimate the generalization error of a classifier trained on the available data with 10-fold-cross validation and leave-one-out cross-validation. Which one has a bigger bias and a bigger variance? Explain your response. You need not carry out additional experiments, only explain what you should expect to obtain according to what you have seen in the lectures.